<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 12:58:07
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 52.39 K (0.37%)
Current PnL: -19.88 L (-13.24%)
CY Booked + Current PnL: -8.44 L (-5.62%)
-------------------
Total profit:  1.80 L
Total loss:  -21.68 L
-------------------
Total Booked + Current PnL: 18.54 L (15.06%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.39 L (61.67%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.25,118651.0,18147.0,10963.0,-2.47,18.06,9.24,28.96,79.0,1.66,0.84,58.16,MH,ATH,METALS
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,213282.0,8782.0,14503.0,2.89,4.29,6.80,11.39,10.0,0.61,1.51,13.63,X40N,ATH,PHARMA
14,BLUESTARCO,2080.00,11.93,56.0,X-MC,3.17,193100.0,28430.0,14907.0,2.05,17.26,7.72,26.31,89.0,1.91,1.37,26.63,X40N,NTT,AC
84,VOLTAS,1530.00,2.00,63.0,X-MC,1.11,213975.0,22233.0,15535.0,3.22,11.60,7.26,19.69,99.0,1.43,1.52,19.18,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.01,83192.0,-17585.0,17678.0,-0.31,-17.45,21.25,0.09,245.0,-0.99,0.59,9.30,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-7.87,36.0,H-SC,6.27,109746.0,-15551.0,74474.0,3.25,-12.41,67.86,47.03,141.0,-0.21,0.78,37.41,MH,ATH,POWER
84,VOLTAS,1530.00,2.00,63.0,X-MC,1.11,213975.0,22233.0,15535.0,3.22,11.60,7.26,19.69,99.0,1.43,1.52,19.18,XY25,NTT,AC
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,213282.0,8782.0,14503.0,2.89,4.29,6.80,11.39,10.0,0.61,1.51,13.63,X40N,ATH,PHARMA
53,QUESS,986.00,-46.01,36.0,X-SC,37.75,53622.0,-11384.0,162314.0,2.88,-17.51,302.70,232.18,198.0,-0.07,0.38,2.88,XY24,NTT,MISC
21,COFFEEDAY,80.00,-43.66,55.0,L-SC,29.28,84192.0,-29357.0,69408.0,2.69,-25.85,82.44,35.27,268.0,-0.42,0.60,106.06,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215042.0,9966.0,102597.0,-3.49,4.86,47.71,54.89,52.0,0.10,1.52,30.13,X5K,ATH,METALS
60,SAIL,228.00,44.35,51.0,M-MC,12.67,228010.0,3048.0,164600.0,-2.97,1.35,72.19,74.53,192.0,0.02,1.62,34.05,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-23.71,46.0,H-MC,6.63,103113.0,-28722.0,70942.0,-2.62,-21.79,68.80,32.03,98.0,-0.40,0.73,15.50,OX40N,NTT,IT
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.25,118651.0,18147.0,10963.0,-2.47,18.06,9.24,28.96,79.0,1.66,0.84,58.16,MH,ATH,METALS
0,5PAISA,593.00,107.93,32.0,H-SC,7.81,141003.0,-77493.0,105682.0,-1.82,-35.47,74.95,12.90,153.0,-0.73,1.00,7.33,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,63.0,H-SC,13.78,131463.0,-11016.0,30539.0,1.00,-7.73,23.23,13.70,163.0,-0.36,0.93,55.16,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.01,83192.0,-17585.0,17678.0,-0.31,-17.45,21.25,0.09,245.0,-0.99,0.59,9.30,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,51.0,H-SC,1.94,222210.0,-47457.0,83795.0,0.63,-17.60,37.71,13.47,138.0,-0.57,1.57,12.99,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.71,46.0,H-MC,6.63,103113.0,-28722.0,70942.0,-2.62,-21.79,68.80,32.03,98.0,-0.40,0.73,15.50,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137471.0,-38432.0,80448.0,-0.76,-21.85,58.52,23.89,149.0,-0.48,0.97,18.72,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,51.0,M-MC,12.67,228010.0,3048.0,164600.0,-2.97,1.35,72.19,74.53,192.0,0.02,1.62,34.05,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215042.0,9966.0,102597.0,-3.49,4.86,47.71,54.89,52.0,0.10,1.52,30.13,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.70,154814.0,3869.0,105583.0,1.34,2.56,68.20,72.51,53.0,0.04,1.10,8.74,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,57.0,M-LC,9.18,158225.0,17385.0,27104.0,0.97,12.34,17.13,31.59,66.0,0.64,1.12,42.10,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195218.0,15410.0,97004.0,0.91,8.57,49.69,62.52,88.0,0.16,1.38,45.58,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.25,118651.0,18147.0,10963.0,-2.47,18.06,9.24,28.96,79.0,1.66,0.84,58.16,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195218.0,15410.0,97004.0,0.91,8.57,49.69,62.52,88.0,0.16,1.38,45.58,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215042.0,9966.0,102597.0,-3.49,4.86,47.71,54.89,52.0,0.10,1.52,30.13,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.70,154814.0,3869.0,105583.0,1.34,2.56,68.20,72.51,53.0,0.04,1.10,8.74,XR,NTT,IT
37,INDIAMART,4810.62,-54.27,44.0,H-SC,8.74,127544.0,4208.0,127416.0,-0.24,3.41,99.90,106.72,119.0,0.03,0.90,27.59,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DABUR,735.00,-12.30,35.0,X-MC,3.75,237238.0,-10830.0,119236.0,0.77,-4.37,50.26,43.70,102.0,-0.09,1.68,9.45,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.16,36.0,X-LC,5.55,206668.0,-45100.0,98312.0,0.52,-17.91,47.57,21.13,27.0,-0.46,1.46,11.17,X40,ATH,PAINTS
53,QUESS,986.00,-46.01,36.0,X-SC,37.75,53622.0,-11384.0,162314.0,2.88,-17.51,302.70,232.18,198.0,-0.07,0.38,2.88,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221770.0,-41595.0,154951.0,0.41,-15.79,69.87,43.04,84.0,-0.27,1.57,2.96,XY25,ATH,FMCG
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.56,81677.0,-63483.0,139414.0,-0.79,-43.73,170.69,52.31,136.0,-0.46,0.58,9.33,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-4.76,46.0,X-MC,0.51,236184.0,-11547.0,86609.0,1.90,-4.66,36.67,30.30,92.0,-0.13,1.67,8.01,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,2.00,63.0,X-MC,1.11,213975.0,22233.0,15535.0,3.22,11.60,7.26,19.69,99.0,1.43,1.52,19.18,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,43.0,X-LC,1.59,246586.0,-29937.0,140012.0,-0.26,-10.83,56.78,39.81,54.0,-0.21,1.75,18.18,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.54,48.0,H-LC,1.67,158230.0,-27865.0,75349.0,-0.43,-14.97,47.62,25.51,15.0,-0.37,1.12,16.60,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-19.41,46.0,X-MC,1.84,138802.0,2669.0,28454.0,-0.37,1.96,20.50,22.86,91.0,0.09,0.98,14.82,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.75,53622.0,-11384.0,162314.0,2.88,-17.51,302.70,232.18,198.0,-0.07,0.38,2.88,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221770.0,-41595.0,154951.0,0.41,-15.79,69.87,43.04,84.0,-0.27,1.57,2.96,XY25,ATH,FMCG
83,VBL,671.64,-20.64,42.0,X-LC,10.32,285888.0,-29954.0,142630.0,0.49,-9.48,49.89,35.67,5.0,-0.21,2.03,3.06,X40N,ATH,FMCG
42,ITC,452.00,-39.19,45.0,X-LC,2.40,196749.0,-3389.0,24279.0,0.61,-1.69,12.34,10.44,4.0,-0.14,1.39,3.78,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.48,39.0,X-MC,4.96,180400.0,-20106.0,117260.0,-0.08,-10.03,65.00,48.45,90.0,-0.17,1.28,4.03,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.75,53622.0,-11384.0,162314.0,2.88,-17.51,302.70,232.18,198.0,-0.07,0.38,2.88,XY24,NTT,MISC
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.56,81677.0,-63483.0,139414.0,-0.79,-43.73,170.69,52.31,136.0,-0.46,0.58,9.33,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.87,43.0,X-SC,4.42,97293.0,-31377.0,76677.0,-0.01,-24.39,78.81,35.20,219.0,-0.41,0.69,11.67,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,51.0,X-SC,1.44,108242.0,6238.0,37690.0,0.91,6.12,34.82,43.06,122.0,0.17,0.77,24.28,X40N,ATH,MISC
33,HONAUT,58357.33,-22.51,48.0,X-SC,2.13,109260.0,-18678.0,65807.0,0.50,-14.60,60.23,36.84,143.0,-0.28,0.77,11.86,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,51.0,X-LC,12.16,284895.0,-61061.0,129314.0,-1.01,-17.65,45.39,19.73,1.0,-0.47,2.02,4.93,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,322184.0,9296.0,162381.0,0.22,2.97,50.40,54.87,3.0,0.06,2.28,9.82,X40,BTT,IT
42,ITC,452.00,-39.19,45.0,X-LC,2.40,196749.0,-3389.0,24279.0,0.61,-1.69,12.34,10.44,4.0,-0.14,1.39,3.78,X40,NTT,FMCG
83,VBL,671.64,-20.64,42.0,X-LC,10.32,285888.0,-29954.0,142630.0,0.49,-9.48,49.89,35.67,5.0,-0.21,2.03,3.06,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,45.0,H-LC,7.50,248352.0,-13267.0,132471.0,0.53,-5.07,53.34,45.57,7.0,-0.10,1.76,5.68,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,55.0,L-SC,29.28,84192.0,-29357.0,69408.0,2.69,-25.85,82.44,35.27,268.0,-0.42,0.60,106.06,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,42.0,H-SC,10.37,90675.0,-7305.0,28907.0,0.35,-7.46,31.88,22.05,152.0,-0.25,0.64,31.80,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.25,118651.0,18147.0,10963.0,-2.47,18.06,9.24,28.96,79.0,1.66,0.84,58.16,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,63.0,H-SC,13.78,131463.0,-11016.0,30539.0,1.00,-7.73,23.23,13.70,163.0,-0.36,0.93,55.16,OX40N,NTT,BANKS
26,GREENPANEL,537.00,233.33,59.0,M-SC,4.68,148791.0,-34287.0,113260.0,-0.05,-18.73,76.12,43.14,230.0,-0.30,1.05,39.96,XY24,NTT,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.44,147510.0,-1710.0,145887.0,1.76,-1.15,98.90,96.62,208.0,-0.01,1.05,59.64,XY25,NTT,FINANCE
69,SURYODAY,240.00,66.67,66.0,H-SC,9.04,153581.0,-25490.0,91212.0,1.33,-14.23,59.39,36.71,135.0,-0.28,1.09,52.38,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215042.0,9966.0,102597.0,-3.49,4.86,47.71,54.89,52.0,0.10,1.52,30.13,X5K,ATH,METALS
26,GREENPANEL,537.00,233.33,59.0,M-SC,4.68,148791.0,-34287.0,113260.0,-0.05,-18.73,76.12,43.14,230.0,-0.30,1.05,39.96,XY24,NTT,MISC
58,REPCOHOME,880.00,-57.53,65.0,H-SC,2.80,242689.0,-43816.0,301153.0,1.80,-15.29,124.09,89.82,134.0,-0.15,1.72,28.40,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.45
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
LC    32.30
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.60
X40N     11.29
XR       11.01
XY25     10.45
AR        8.16
OX40N     7.60
X200      1.75
MH        1.62
X5K       1.52
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.48
X-LC    22.92
X-MC    17.20
M-SC    12.51
H-LC     4.63
H-MC     4.60
X-SC     4.53
M-LC     3.74
M-MC     1.62
L-SC     1.46
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-5.80,40.67
IT,12.85,-15.94,78.73
FINANCE,10.50,-13.91,68.92
MISC,6.90,-17.79,83.05
BANKS,6.41,-10.33,68.37
PAINTS,5.48,-19.21,36.99
ELECTRICAL,5.32,-9.49,47.90
INSURANCE,3.78,-3.61,42.35
AC,3.58,4.00,12.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3215684.0,22
XR,1356383.0,14
AR,1177612.0,9
X40,776898.0,10
X40N,642968.0,9
OX40N,573713.0,10
XY25,463738.0,7
SR,264697.0,2
X5K,102597.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404224.0,25
M-SC,1277200.0,15
X-MC,1107534.0,12
X-LC,1005898.0,13
X-SC,540931.0,6
H-MC,359684.0,4
H-LC,286748.0,3
L-SC,255572.0,3
M-LC,235284.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1195676.0      6
           AR         805713.0      5
           XR         795755.0      7
M-SC       XY24       689042.0      6
X-MC       XY24       589767.0      4
X-LC       X40        485107.0      5
X-SC       X40N       301940.0      4
H-SC       OX40N      267909.0      4
           SR         264697.0      2
X-MC       X40        215114.0      4
X-LC       X40N       208861.0      3
H-LC       AR         207820.0      2
X-LC       XY24       206406.0      2
H-MC       XY24       180775.0      1
X-MC       XY25       170486.0      2
L-SC       XR         167358.0      2
M-MC       XY24       164600.0      1
M-SC       AR         164079.0      2
X-SC       XY24       162314.0      1
M-SC       OX40N      146648.0      4
           XY25       145887.0      1
X-MC       X40N       132167.0      2
M-SC       XR         131544.0      2
M-LC       XR         105583.0      1
X-LC       XY25       105524.0      3
M-LC       X5K        102597.0      1
H-MC       XR          97004.0      1
L-SC       OX40N       88214.0      1
H-LC       X200        78928.0      1
X-SC       X40         76677.0      1
H-SC       MH          74474.0      1
H-MC       OX40N       70942.0      1
L-MC       XR          59139.0      1
L-LC       XY25        41841.0      1
M-LC       XY24        27104.0      1
H-MC       MH          10963.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
